In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder
import shap


df=pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
df.head()

: 

In [ ]:
print(df.head())

In [ ]:
print(df.info())

In [ ]:
print(df.describe())

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.duplicated().sum())

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
print(df.duplicated().sum())

In [ ]:
import matplotlib.pyplot as plt
df.head()
plt.figure(figsize=(12,6))
df.boxplot(rot=90)
plt.title("Detection des outliers")
plt.show()

In [ ]:
from scipy.stats import zscore 
z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).sum(axis=0)
print("Nombre d'outliers detectes par case :")
print(outliers)

In [ ]:
duplicates = df[df.duplicated(keep=False)]

In [ ]:
print(df.duplicated().sum()) 

In [ ]:
duplicates = df[df.duplicated(keep=False)]

print("Lignes dupliquées :")
print(duplicates)

In [ ]:
for col in df.columns:
    duplicate_col = df.duplicated(subset=[col], keep=False)
    if duplicate_col.any():
        print(f"Doublons détectés dans la colonne: {col}")
        print(df.loc[duplicate_col, [col]])


In [ ]:

df_numeric = df.select_dtypes(include=['number'])


Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1


outliers_iqr = ((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))).sum()

print("Outliers détectés par la méthode IQR :")
print(outliers_iqr)


In [ ]:
df = df.drop_duplicates()
print(df.shape)  

In [ ]:

df_numeric = df.select_dtypes(include=['number'])

print(df_numeric.columns)


plt.figure(figsize=(12, 6))


sns.boxplot(data=df_numeric[['Age', 'Height', 'Weight', 'NCP']])

plt.xticks(rotation=45)
plt.title("Boxplot of Outlier-Affected Columns")
plt.show()


In [ ]:
Q1_NCP = df['NCP'].quantile(0.25)
Q3_NCP = df['NCP'].quantile(0.75)
IQR_NCP = Q3_NCP - Q1_NCP


lower_bound = max(Q1_NCP - 1.5 * IQR_NCP, 1) 
upper_bound = min(Q3_NCP + 1.5 * IQR_NCP, 5) 


df_capped = df.copy()
df_capped['NCP'] = df['NCP'].clip(lower=lower_bound, upper=upper_bound)


In [ ]:
df_filtered = df[(df['NCP'] >= 1) & (df['NCP'] <= 5)]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


class_distribution = df['NObeyesdad'].value_counts(normalize=True) * 100  # Convert to percentage

plt.figure(figsize=(10, 6))
sns.barplot(x=class_distribution.index, y=class_distribution.values, palette="viridis")


plt.xlabel("Obesity Levels")
plt.ylabel("Percentage (%)")
plt.title("Distribution of Obesity Levels")
plt.xticks(rotation=45) 

plt.show()



In [ ]:
target_col = 'NObeyesdad'  

class_distribution = df[target_col].value_counts()

print("Class Distribution:\n", class_distribution)
plt.figure(figsize=(10, 6))
sns.barplot(x=class_distribution.index, y=class_distribution.values, palette="viridis")
plt.xlabel("Obesity Level Classes")
plt.ylabel("Count")
plt.title("Distribution of Obesity Classes")
plt.xticks(rotation=45)  
plt.show()

In [ ]:

class_distribution = df['NObeyesdad'].value_counts(normalize=True) * 100
print(class_distribution)


In [ ]:
from imblearn.over_sampling import SMOTE
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('NObeyesdad') 

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le 

target_col = "NObeyesdad"
X = df.drop(columns=[target_col])
y = df[target_col]

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

from collections import Counter
print("New class distribution:", Counter(y_resampled))

plt.figure(figsize=(10, 6))
sns.barplot(x=pd.Series(y_resampled).value_counts().index,
            y=pd.Series(y_resampled).value_counts().values,
            palette="viridis")
plt.xlabel("Obesity Level Classes")
plt.ylabel("Count")
plt.title("Balanced Distribution of Obesity Classes After SMOTE")
plt.xticks(rotation=45)
plt.show()

In [ ]:

df_numeric = df.select_dtypes(include=['number'])
correlation_matrix = df_numeric.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

In [ ]:
correlation_threshold = 0.8
strong_correlations = {}
for col in correlation_matrix.columns:
    strong_corrs = correlation_matrix[col][(correlation_matrix[col] > correlation_threshold) & (correlation_matrix[col] < 1)].index.tolist()
    if strong_corrs:
        strong_correlations[col] = strong_corrs
strong_correlations

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import pandas as pd

df.head()

X = df.drop(columns=['NObeyesdad']).copy()  
y = df['NObeyesdad']
categorical_cols = X.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X.loc[:, col] = le.fit_transform(X[col]) 
    label_encoders[col] = le  
scaler = StandardScaler()
X.loc[:, X.select_dtypes(include=['number']).columns] = scaler.fit_transform(X.select_dtypes(include=['number']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:

import pandas as pd
import numpy as np

import shap
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report
df.head()

categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


le_target = LabelEncoder()
df['NObeyesdad'] = le_target.fit_transform(df['NObeyesdad'])
X = df.drop(columns=['NObeyesdad', 'Height', 'Weight']) 
y = df['NObeyesdad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
numerical_features = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])
models = {
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier()}

model_performance = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    model_performance[name] = accuracy
    print(f"Model: {name}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)
best_model_name = max(model_performance, key=model_performance.get)
best_model = models[best_model_name]

print(f"Best Model: {best_model_name} with Accuracy: {model_performance[best_model_name]:.4f}")
if best_model_name == "Random Forest":
    feature_importance = best_model.feature_importances_
elif best_model_name == "XGBoost":
    feature_importance = best_model.feature_importances_
elif best_model_name == "LightGBM":
    feature_importance = best_model.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(X.columns, feature_importance)
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title(f'Feature Importance for {best_model_name}')
plt.show()

In [ ]:
def optimize_memory(df):
    for col in df.columns:
        col_type = df[col].dtype
        if pd.api.types.is_integer_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif pd.api.types.is_float_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif pd.api.types.is_object_dtype(col_type):
            df[col] = df[col].astype('category')
    
    return df

def demonstrate_memory_optimization(df):
    before_memory = df.memory_usage(deep=True)
    print("Memory usage before optimization:")
    print(before_memory)
    df_optimized = optimize_memory(df)
    after_memory = df_optimized.memory_usage(deep=True)
    print("\nMemory usage after optimization:")
    print(after_memory)
    return before_memory, after_memory
before_memory, after_memory = demonstrate_memory_optimization(df)

before_memory, after_memory

In [ ]:
import shap
import joblib


explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test)


for i in range (shap_values.shape[2]):
    print(f"Shape of SHAP values for class {i}: {shap_values[:,:,i].shape}")
    shap.summary_plot(shap_values[:,:,i], X_test)
    shap.dependence_plot('Age', shap_values[:,:,i], X_test)


MODEL_PATH = "best_model.pkl"
SCALER_PATH = "scaler.pkl"
ENCODERS_PATH = "label_encoders.pkl"
joblib.dump(best_model, MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)
joblib.dump(label_encoders, ENCODERS_PATH)
